<a href="https://colab.research.google.com/github/tlsehdgns1999/tlsehdgns1999/blob/main/lg_aimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import warnings
import numpy as np
def deprecated_function():
    warnings.warn("This function is deprecated.", DeprecationWarning)

warnings.filterwarnings("ignore", category=DeprecationWarning)

deprecated_function()

In [2]:
pd.set_option('display.max_seq_items', None)

In [3]:
path = '/content/drive/MyDrive/lgaimers/train.csv'
path_sales = '/content/drive/MyDrive/lgaimers/sales.csv'
path_info = '/content/drive/MyDrive/lgaimers/product_info.csv'
path_key = '/content/drive/MyDrive/lgaimers/brand_keyword_cnt.csv'

In [4]:
data = pd.read_csv(path) # 일별 판매량
data_sales = pd.read_csv(path_sales) # 실제 판매되고 있는 고유 ID
data_info = pd.read_csv(path_info) # 제품 : 제품 코드   /   제품특성 : 제품 특성 데이터(Text)
data_key = pd.read_csv(path_key) # 브랜드 : 브랜드 코드 / 브랜드의 연관키워드 언급량을 정규화한 일별 데이터
data_sub = pd.read_csv('/content/drive/MyDrive/lgaimers/sample_submission.csv')

In [5]:
# 날짜 컬럼들을 추출합
date_columns = data.columns[6:]  # '2022-01-01'부터 날짜 컬럼들이 시작됨

# 날짜를 기준으로 data_sales를 data
merged_df = pd.merge(data_sales, data, on=['ID', '제품', '대분류', '중분류', '소분류', '브랜드'], how='inner')

# 개당 가격 계산
for date_column in date_columns:
    merged_df[date_column + '_가격'] = merged_df[date_column + '_x'] / merged_df[date_column + '_y']

merged_df.head()

<ipython-input-5-1771e4eec481>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[date_column + '_가격'] = merged_df[date_column + '_x'] / merged_df[date_column + '_y']
<ipython-input-5-1771e4eec481>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[date_column + '_가격'] = merged_df[date_column + '_x'] / merged_df[date_column + '_y']
<ipython-input-5-1771e4eec481>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01_x,2022-01-02_x,2022-01-03_x,2022-01-04_x,...,2023-03-26_가격,2023-03-27_가격,2023-03-28_가격,2023-03-29_가격,2023-03-30_가격,2023-03-31_가격,2023-04-01_가격,2023-04-02_가격,2023-04-03_가격,2023-04-04_가격
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,NaN,NaN,NaN,22400.0,22400.0,22400.0,NaN,NaN,22400.0,NaN
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data['제품'].nunique()

15840

In [7]:
merged_df = merged_df.drop(merged_df.filter(regex='_x$').columns, axis=1)

# '_y'로 끝나는 열을 삭제하지 않으면서 '_가격'으로 끝나는 열
filtered_columns = [col for col in merged_df.columns if '_y' not in col or '_가격' in col]
merged_df = merged_df[['ID', '제품', '대분류', '중분류', '소분류', '브랜드'] + filtered_columns]

In [8]:
pd.set_option('display.max_seq_items', None)

In [9]:
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()] # 중복된 컬럼 제거

In [10]:
date_columns = merged_df.columns[6:]  # 날짜 컬럼들은 6번째 열부터 시작

# 각 행별로 날짜 컬럼들의 값을 리스트로 저장
merged_df['제품가격'] = merged_df[date_columns].apply(lambda row: row.dropna().tolist(), axis=1)

In [11]:
# 제품별로 가장 빈도수가 높은 값을 선택하여 제품 가격 열을 갱신
def most_common_value(series):
    return series.value_counts().idxmax()

product_prices = merged_df[merged_df.columns[6:]].apply(most_common_value, axis=1)

# 제품 코드 열을 포함한 데이터프레임 생성
price = pd.concat([merged_df['제품'], product_prices], axis=1)
price.columns = ['제품', '제품가격']

In [37]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15890 entries, 0 to 15889
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품      15890 non-null  object
 1   제품가격    15890 non-null  object
dtypes: object(2)
memory usage: 888.5+ KB


In [38]:
price['제품가격'] = price['제품가격'].apply(lambda x: int(round(x)) if pd.notna(x) else x)

In [39]:
data_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12778 entries, 0 to 12777
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품      12778 non-null  object
 1   제품특성    12778 non-null  object
dtypes: object(2)
memory usage: 199.8+ KB


In [40]:
df_info = data_info.merge(price.drop_duplicates(subset=['제품']), on='제품', how='left')

In [41]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12778 entries, 0 to 12777
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품      12778 non-null  object
 1   제품특성    12778 non-null  object
 2   제품가격    10442 non-null  object
dtypes: object(3)
memory usage: 399.3+ KB


In [42]:
df_info['제품가격']

0        21760
1        43650
2        16600
3        16900
4        38000
         ...  
12773      NaN
12774      NaN
12775       []
12776      NaN
12777      NaN
Name: 제품가격, Length: 12778, dtype: object

In [44]:
df_info['제품가격'] = df_info['제품가격'].apply(lambda x: int(round(x)) if pd.notna(x) else x)

In [66]:
df_info['제품가격']

0        21760
1        43650
2        16600
3        16900
4        38000
         ...  
12773        0
12774        0
12775       []
12776        0
12777        0
Name: 제품가격, Length: 12778, dtype: object

In [65]:
df_info['제품가격'] = df_info['제품가격'].replace(' []', 0)
df_info['제품가격'] = df_info['제품가격'].fillna(0)

In [24]:
df_info # 제품 특성과 가격

,제품,제품특성,제품가격
0,B002-03509-00001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...,21760.0
1,B002-02376-00001,700mg x 28정,43650.0
2,B002-03735-00001,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...,16600.0
3,B002-03735-00003,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 한 번 1일 총 섭...,16900.0
4,B002-02769-00001,HCA:900mg 영양소 원료명(식약처고시):비타민B1 영양소 원료명(식약처고시):...,38000.0
...,...,...,...
12773,B002-01994-00001,형태:액상형 구성:리필 등급:1종 용도:식기 용도:과일 용도:야채 용도:조리기구 용...,NaN
12774,B002-02600-00002,헤어타입:모든 모발용 주요제품특징:머릿결개선 세부제품특징:촉촉함(수분공급) 세부제품...,NaN
12775,B002-02372-00095,사용대상:여성용 타입:일자형 흡수량:20ml :1팩 32개,[]
12776,B002-01318-00002,피부타입:모든피부용 향계열:무향 주요제품특징:촉촉함(수분공급) 주요제품특징:풍부한 ...,NaN


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [13]:
# 레이블 인코딩
label_encoder = LabelEncoder()
data_info["클러스터"] = label_encoder.fit_transform(data_info["제품특성"])

# TfidfVectorizer로 텍스트 데이터 벡터화
vectorizer = TfidfVectorizer(max_features=5000)  # 상황에 따라 max_features 값 조정
X = vectorizer.fit_transform(data_info["제품특성"])

# K-Means 클러스터링
num_clusters = 11  # 클러스터 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data_info["클러스터"] = kmeans.fit_predict(X)

# 클러스터링 결과 데이터 프레임에 추가
cluster_df = pd.DataFrame({'제품': data_info['제품'], '제품특성': data_info['제품특성'], '클러스터': data_info['클러스터']})

# 클러스터링 결과 출력
print(cluster_df)

# 각 클러스터에 속하는 제품 출력
for cluster_id in range(num_clusters):
    cluster_products = cluster_df[cluster_df["클러스터"] == cluster_id]["제품"]
    print(f"클러스터 {cluster_id}에 속하는 제품:\n", cluster_products.values)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                     제품                                               제품특성  \
0      B002-03509-00001  제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...   
1      B002-02376-00001                                        700mg x 28정   
2      B002-03735-00001  제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...   
3      B002-03735-00003  제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 한 번 1일 총 섭...   
4      B002-02769-00001  HCA:900mg 영양소 원료명(식약처고시):비타민B1 영양소 원료명(식약처고시):...   
...                 ...                                                ...   
12773  B002-01994-00001  형태:액상형 구성:리필 등급:1종 용도:식기 용도:과일 용도:야채 용도:조리기구 용...   
12774  B002-02600-00002  헤어타입:모든 모발용 주요제품특징:머릿결개선 세부제품특징:촉촉함(수분공급) 세부제품...   
12775  B002-02372-00095                   사용대상:여성용 타입:일자형 흡수량:20ml :1팩 32개   
12776  B002-01318-00002  피부타입:모든피부용 향계열:무향 주요제품특징:촉촉함(수분공급) 주요제품특징:풍부한 ...   
12777  B002-02473-00064  최소연령:4개월 단계별:2단계 종류:일반분유 제품형태:분말 포장형태:캔 용량:800...   

       클러스터  
0         2  
1         6  
2         4  
3      

In [20]:
cluster_df

,제품,제품특성,클러스터
0,B002-03509-00001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...,2
1,B002-02376-00001,700mg x 28정,6
2,B002-03735-00001,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...,4
3,B002-03735-00003,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 한 번 1일 총 섭...,2
4,B002-02769-00001,HCA:900mg 영양소 원료명(식약처고시):비타민B1 영양소 원료명(식약처고시):...,4
...,...,...,...
12773,B002-01994-00001,형태:액상형 구성:리필 등급:1종 용도:식기 용도:과일 용도:야채 용도:조리기구 용...,9
12774,B002-02600-00002,헤어타입:모든 모발용 주요제품특징:머릿결개선 세부제품특징:촉촉함(수분공급) 세부제품...,10
12775,B002-02372-00095,사용대상:여성용 타입:일자형 흡수량:20ml :1팩 32개,6
12776,B002-01318-00002,피부타입:모든피부용 향계열:무향 주요제품특징:촉촉함(수분공급) 주요제품특징:풍부한 ...,1
